In [ ]:
!pip install langdetect html2text

In [ ]:
!python -m pip install mysql-connector-python

In [3]:
import pandas as pd
import html2text
import pickle
import mysql.connector
import cleaning_functions
import connection_database

## Connection to the database

## Get data

In [5]:
mydb = connection_database.connect_db()
cursor = mydb.cursor()

command = """SELECT emails.id, body, organization_id FROM emails 
JOIN tasks ON emails.task_id=tasks.id 
WHERE organization_id IN 
(SELECT id FROM organizations WHERE default_pgrefid IS NULL)
AND emails.from NOT LIKE '%@closure-services.nl%'
AND emails.is_auto_receival=0
AND emails.subject NOT LIKE "%ntvangst bevestiging%" 
AND emails.created_at > 20181231"""

cursor.execute(command)
table_rows = cursor.fetchall()
emails = pd.DataFrame(table_rows, columns=cursor.column_names)

## Convert HTML to text

In [7]:
emails["body_text"] = emails["body"].apply(lambda x: html2text.html2text(x))
emails = emails[["id", "body_text", "organization_id"]]

## Remove history of previous emails

In [8]:
emails["no_history"] = emails["body_text"].apply(cleaning_functions.remove_history)

## Keep only before greetings

In [9]:
emails["before_regards"] = emails["no_history"].apply(cleaning_functions.remove_after_regards)

## Remove url adresses

In [10]:
emails["before_regards_clean"] = emails["before_regards"].apply(cleaning_functions.remove_url_emails)

## Remove begining of the email (before dear)

In [11]:
emails["only_body"] = emails["before_regards_clean"].apply(cleaning_functions.remove_before_dear)

## Remove space at the end

In [12]:
emails["only_body"] = emails["only_body"].apply(cleaning_functions.remove_space_end)

## Keep only Dutch emails

In [13]:
emails["is_Dutch"] = emails["only_body"].apply(cleaning_functions.check_Dutch)
emails = emails.loc[emails['is_Dutch'] == 1]
emails = emails[["id", "only_body", "organization_id"]]

## Save cleaned data

In [28]:
with open('dutch_cleaned_emails_id_body_org.pkl', 'wb') as f:
    pickle.dump(emails, f)